In [ ]:
from line_solver import *
import numpy as np
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
#Advanced Fork-Join Example 12#Complex fork-join with 2 parallel branches
model = Network('model')

In [ ]:
num_branches = 2 #Create complex fork-join network
source = Source(model, 'Source')
fork = Fork(model, 'Fork')
fork.setTasksPerLink(num_branches) #Create multiple parallel queues

queues = []
for i in range(num_branches):
    queue = Queue(model, f'Queue{i+1}', SchedStrategy.PS)
queues.append(queue)

join = Join(model, 'Join', fork)
sink = Sink(model, 'Sink')

jobclass = OpenClass(model, 'Class1')

In [ ]:
#Service configurations
source.setArrival(jobclass, Exp(0.5))
for i, queue in enumerate(queues):
    queue.setService(jobclass, Exp(1.0 + i * 0.1))

In [ ]:
# Fork-join routing
P = model.initRoutingMatrix()
P.set(jobclass, jobclass, source, fork, 1.0)
for queue in queues:
    P.set(jobclass, jobclass, fork, queue, 1.0)
P.set(jobclass, jobclass, queue, join, 1.0)
P.set(jobclass, jobclass, join, sink, 1.0)
model.link(P)

In [ ]:
#Solver
solver = SolverJMT(model, 'samples', 15000)
print(solver.getAvgTable())